In [333]:
import os
import sys
import time # add
import csv # add
import heapq
import numpy as np
import math
import statistics 
import itertools #import combinations ### added
from collections import namedtuple

In [313]:
class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to its children. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""    
    
    def __init__(self, parent = None, x= None, index= None, value= None, estimate= None, room= None):
        """
            action: a list for valid children
        """ 
        self.x = x # decision variable
        self.index = index # item index
        self.value = value
        self.estimate = estimate
        self.room = room 

        self.state =''.join([str(self.index), str(self.x)])
        self.parent = parent
        self.depth = 0
        if parent:
            self.depth = parent.depth+1    
            self.state = parent.state + self.state
            
    def __repr__(self):
        """Use by calling repr(node)"""
        return "Node(" + repr(self.index) + "," + repr(self.x) + "," +repr(self.value) + ","\
    + repr(self.estimate) + "," + repr(self.room) + ")" 
            
    def __lt__(self, node):
        return self.estimate < node.value
    
    def expand(self, items, fractions):
        """
            fractions is 1 by len(items) containing fractions of item.values, e.g.,
            1 for select all item and 0.2 for select 0.2 of the items weight and value
        """
        "List the nodes reachable in one step from this node."
        if self.index >= len(items)-1:
            return None
        
        children = []
        for x in range(2):
            if x == 0: # not select this child
                # assuming item[0].index 0 corresponds to fractions[0]
                estimate = self.estimate - fractions[self.index+1]*items[self.index+1].value
                child = self.child_node(x, self.index+1, self.value, estimate, self.room) 
                children.append(child)
            elif self.room-items[self.index+1].weight >=0: # select this child if it can fit
                child = self.child_node(x, self.index+1, 
                                self.value+items[self.index+1].value,
                                self.estimate, self.room-items[self.index+1].weight)
                children.append(child)
                
        return children
    
    def child_node(self,  x, index, value, estimate, room):
        return Node(self, x, index, value, estimate, room)
    
    def path(self):
        "Return a loist of nodes forming the path from the root to this node"
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def solution(self):
        "Return items selected along the path from root to this node"
        return self.value, [node.x for node in self.path()[1:]]
    def __eq__(self, other):
        return isinstance(other, Node) and self.x == other.x and self.index == other.index
    def __has__(self):
        return hash((self.x, self.index))

In [314]:
class PriortyBinaryNode:

    """A node in a binary search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to its children. The child node with higher $value$
    is on the left and the lower one on the right.
    """      
    
    def __init__(self, parent = None, x= -1, index= -1, value= None, 
                 estimate= None, room= None, right_count = 0):
        """
            x: a decision variable: 1 for selected, 0 otherwise
            index: node index
            item : a item number: for n items, 0,1,....,n-1
            estimate: optimistic evaluation of possible value for this node
            value: actual value
            room: remaining capacity
            state: use as a key in a dictionary keeping track of already computed node
                   it is a string combination of index and x
            depth: the depth in which the node resides
            left: its child with higher value than the other child
            right: its child with lower value than the other child
        """ 
        self.x = x # decision variable
        self.index = index # 
        self.item = None
        self.value = value
        self.estimate = estimate
        self.room = room 
        self.parent = parent     
        self.left, self.right = None, None
        self.number_of_mistake = 0
        self.depth = 0
        self.state = ''.join([str(self.depth), str(self.index), str(self.x)])
        if self.parent:
            self.depth += self.parent.depth
            self.state = ''.join([str(parent.depth), str(parent.index), str(parent.x)])+self.state
            
    def set_number_of_mistake(self, n_mistakes):
        if self.number_of_mistake > 0:
            return False
        else: 
            self.number_of_mistake = n_mistakes
            return True
        
    def set_item(self, items):
        if self.index >-1:
            self.item = items[self.index].index
            
    def set_depth(self, depth):
        self.depth = depth
        
    def __repr__(self):
        """Use by calling repr(node)"""
        return "Node(" + repr(self.index) + "," + repr(self.x) + "," +repr(self.value) + ","\
    + repr(self.estimate) + "," + repr(self.room) +  "," + repr(self.item) + "," + repr(self.number_of_mistake) + "," \
    + repr(self.depth) +  ")" 
            
    def __lt__(self, node):
        return self.estimate < node.value
    
    def expand(self, items, fractions):
        """
            Construct direct children of this node
            fractions: a 1 by len(items) vector containing fractions of item.values, e.g.,
                       1 for select all the entire item and 0.2 for select 0.2 of 
                       the items weight and value
        """
        if self.index+1 > len(items)-1:
            self.left, self.right = None, None 
            return [None, None]
        
        child_0, child_1 = None, None
        self.left, self.right = None, None 
            
        for x in range(2):
            if x == 0: # not select this child
                # assuming item[0].index 0 corresponds to fractions[0]
                estimate = self.estimate - fractions[self.index+1]*items[self.index+1].value
                child_0 = self.child_node(x, self.index+1, self.value, estimate, self.room, items, self.depth+1) 
            elif self.room-items[self.index+1].weight >=0: # select this child if it can fit
                child_1 = self.child_node(x, self.index+1, 
                                self.value+items[self.index+1].value,
                                self.estimate, self.room-items[self.index+1].weight, items, self.depth+1)
        if not child_0 and child_1: 
            self.left = child_1
        elif not child_1 and child_0: 
            self.left = child_0
        elif child_0.value > child_1.value: 
            self.left, self.right = child_0, child_1
        else:            
            self.left, self.right = child_1, child_0
                
        return [self.left, self.right]
    
    def child_node(self, x, index, value, estimate, room, items, depth):
        child = PriortyBinaryNode(self, x, index, value, estimate, room)
        child.set_item(items)
        child.set_depth(depth)
        return child

    def path(self):
        "Return a list of nodes forming the path from the root to this node"
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))
    
    def solution(self, depth):
        "Return items selected along the path from root to this node"
        decision_var_values = [0]*depth 
        for node in self.path()[1:]: # omit the root
            decision_var_values[node.item] = node.x
        return int(self.value), decision_var_values 
    
    def __eq__(self, other):
        return isinstance(other, PriortyBinaryNode) and self.x == other.x and self.index == other.index\
    and self.item == other.item and self.value == other.value and self.estimate == other.estimate\
    and self.room == other.room and self.depth == other.depth
    
    def __has__(self):
        return hash((self.x, self.index, self.item,  self.value, self.estimate, self.room, self.depth))


In [315]:
def direct_descendants(curr_node, items, fractions):
    
    if curr_node.index+1 >= len(items): 
        return None, None
    
    curr_node.expand(items, fractions)
    return curr_node.left, curr_node.right
    
def limited_discrepancy_search_probe(best_node, num_mistake, items, fractions, frontiers, explored): 
    
    def is_solution(best_node, node):
        
        #Reach goal with value at best node > optimistic estimate of current node
        if float(best_node.value) > float(node.estimate): 
            return True, best_node
        
        if float(node.value) > float(best_node.value):
            best_node = node
            return False, best_node
        
        return False, None

    def child_recurse(node, num_mistake, explored, frontiers, best_node, end_heap_idx):
        """
            Recursively transverse the tree in depth first fashion
        """
        # return when it hits leave nodes
        if not node: 
            #print('Reach None node in child recurse')
            return explored, frontiers, best_node
        
        # check whether the node is the solution
        flag, result_node = is_solution(best_node, node)
        if result_node: best_node = result_node
        # return when it reach the solution
        if flag: 
            #print('Reach best solution in which node.est < best_node.value in child recurse')
            return explored, frontiers, best_node
            
        # the priority heapq have to contain some elements to use this method
        #if not frontiers:raise ValueError('Input frontiers to child_recurse in Empty')

        # the node should already been put in explored when it gets initiliazed by its parent
        if node.state not in explored: raise ValueError('Node should have already been added to explored')
         
        # the frontiers will be poped until it reaches leave from the previous tree expansion to the left or right&left
        if frontiers:
            end_heap_idx = heapq.nlargest(1, frontiers, key=lambda x:x[0])[0][0]
        
        #print('End hp idx {}, forntiers {}'.format(end_heap_idx, frontiers)) 
        #print('Explored {}'.format(explored)) 
        
        # get the child nodes of this node
        children = node.expand(items, fractions)
        if num_mistake: # we move left if there are mistake
            # visit this child only if num_mistake > the child number of mistake
            if children[1] and num_mistake > children[1].number_of_mistake: 
                children[1].set_number_of_mistake(node.number_of_mistake+1) # increment its mistake
                #print('In child recurse, Cur node {} move right to {} with remaining mistake {}'.format(node, children[1], \
                #                                                                                        num_mistake-1))                
                if children[1].state not in explored: 
                    heapq.heappush(frontiers, (end_heap_idx+1, children[1])) # put into heap for next run
                    explored.add(children[1].state)
                # move to this child
                return child_recurse(children[1], num_mistake-1, explored, frontiers, best_node, None)
            elif not children[1]:
                # we need to move right but the node is a leave node so it has no right child
                #print('Reach the end of the path with none left and right child')
                return explored, frontiers, best_node

        # we have to move left too
        if children[0]:
            children[0].set_number_of_mistake(node.number_of_mistake)
            #print('In child recurse, Cur node {} move left to {} with remaining mistake {}'.format(node, children[0], \
            #                                                                                            num_mistake))            
            if children[0].state not in explored:
                heapq.heappush(frontiers, (end_heap_idx+1, children[0])) 
                explored.add(children[0].state)
            # continue to the left
            return child_recurse(children[0], num_mistake, explored, frontiers, best_node, None)
        
        # if the node has no children, i.e., a leave node
        if not children[0] and not children[1]: return explored, frontiers, best_node
        
    #### end child_recurse function
    
    if len(frontiers) == 1:
        #print('in LDS-Probe, Frontier ', frontiers)
        # for the case where we are supoosed to move only left--> 1 st path from root
        explored, frontiers, best_node = child_recurse(frontiers[0][1], num_mistake, explored, frontiers, best_node,
                                                       frontiers[0][0])
        #print('For first left-most move \nExplored:{} \nFrontiers:{} \nBestnode: {}'.format(explored, frontiers, best_node))
    else:
        end_heap_idx = heapq.nlargest(1, frontiers, key=lambda x:x[0])[0][0]
        heap_element = heapq.heappop(frontiers)
        heap_idx = heap_element[0]
        #print('End heap idx:{}, Heap element idx:{}, node:{}'.format(end_heap_idx, heap_element[0], heap_element[1]))
        while heap_idx <= end_heap_idx:
            node = heap_element[1]
            #print('Explore with {} Mistake:'.format(num_mistake))
            explored, frontiers, best_node = child_recurse(node, 
                                            num_mistake, explored, frontiers, best_node, heap_idx)
            #print('For {} Mistake stating at node{} \nExplored:{} \nFrontiers:{} \nBestnode: {}'.format(num_mistake, node, explored, frontiers, best_node))            

            # keep pop the frontier for each mistake paths, i.e., all 1 mistake paths
            if heap_idx < end_heap_idx: 
                heap_element = heapq.heappop(frontiers)
                heap_idx = heap_element[0]
                #print('Pop to element idx {}, element {}'.format(heap_idx, heap_element))
            else:
                heap_idx = heap_idx+1
            
    return best_node, frontiers, explored



def limited_discrepancy_search(best_estimate, items, fractions, capacity):
    
    # initial root node 
    root = PriortyBinaryNode(parent = None, x= -1, index= -1, value= 0, estimate=best_estimate,
                             room= capacity)
    # initialize best_node as the root node
    best_node = root
    # initialize heap and set used in recursion
    frontiers = []
    heapq.heappush(frontiers, (root.index, root))
    explored = {root.state}
    
    # for each number of mistake, run limited_discrepancy_search_probe
    for num_mistake in range(len(items)):
        
        #print('\nMistake ', num_mistake)
        
        if not frontiers: 
            return best_node
        
        result, frontiers, explored = limited_discrepancy_search_probe(best_node, num_mistake, items,
                                                                fractions, frontiers, explored)

        # if result is better than best_node, we update best_node
        if result and result.value > best_node.value: 
            #print('LDS Best node from LDs-probe to ', result)
            best_node = result
            
        #if frontiers: print('LDS Best node: {}, LDS Computed Frontier: {}'.format(best_node, frontiers))
        #else: print('LDS Best node: {}, LDS Empty Frontiers'.format(best_node))
                
    return best_node

def depth_first_search(fractions, items, capacity, estimate):
    

    root = PriortyBinaryNode(parent = None, x= None, index= -1, value= 0, estimate= estimate, room= capacity)
    frontier = [root]
    explored = set()
    best_node = None
    while frontier:
        node = frontier.pop()
        if not node: return best_node
        if best_node and float(best_node.value) >= float(node.estimate):
            continue 
        if not best_node or node.value > best_node.value:
            best_node = node
        explored.add(node.state)
        children = node.expand(items, fractions)
        if all(children):
            frontier.extend(child for child in children
                            if child.state not in explored and
                            child not in frontier)
    return best_node


In [316]:
def compute_fractions_by_value_density(value_densities, capacity, items):
    """
        Items with high value density are in the first part of the list items
        Items with weight > capacity are removed
    """

    sorted_indices = sorted(range(len(value_densities)), 
                            key=lambda k: value_densities[k], reverse=True)
    
    #print('Sorted Weight ', [items[i].weight for i in sorted_indices])
    #print('Sorted Density ', [value_densities[i] for i in sorted_indices]) 
    #print('Sorted value ', [items[i].value for i in sorted_indices])     

    items = [items[idx] for idx in sorted_indices]
    fractions, room = [0.]*len(items), float(capacity)
    estimate = 0.
    num_omit = 0 # number of items weight > capacity and we remove them
    
    for idx, item in enumerate(items):

        if item.weight > capacity:
            num_omit = num_omit+1
            continue
        elif item.weight <= room: # if item weight less than or equal k, select item
            fractions[idx] = 1.
            estimate = estimate + float(item.value)   
        elif room > 0: # if item weight > k, but is not full yet
            fractions[idx] = float(room)/float(item.weight) # fraction of item that fits
            estimate = estimate + fractions[idx]*float(item.value)

        room = room - fractions[idx]*float(item.weight)

    if num_omit: 
        del items[:num_omit]
        del fractions[:num_omit]

    return items, estimate, fractions


In [373]:
def finite_difference_sorted_weight(sorted_weight, sorted_density, sorted_value, sorted_indices):
    
    #print('At local indx 98, item index {}, weight {}, value {}'.format(sorted_indices[98], sorted_weight[98], \
    #                                                                 sorted_value[98]))    
    max_density_index, max_value_index = -1, -1
    max_density_diff, max_value_diff = 0, 0
        
    finite_diff_weight, finite_diff_density, finite_diff_value = \
    [0.]*len(sorted_weight),[0.]*len(sorted_weight),[0.]*len(sorted_weight)

    for i in range(1, len(sorted_weight),1):
        finite_diff_weight[i] = float(sorted_weight[i]) - float(sorted_weight[i-1])
        finite_diff_value[i] = float(sorted_value[i]) - float(sorted_value[i-1])
        finite_diff_density[i] = float(sorted_density[i]) - float(sorted_density[i-1])
        if max_value_diff < finite_diff_value[i]:
            max_value_diff = finite_diff_value[i]
            max_value_index = i-1
        if max_density_diff < finite_diff_density[i]:
            max_density_diff = finite_diff_density[i]
            max_density_index = i-1
            
    indx = finite_diff_weight.index(max(finite_diff_weight))-1 # due to finite diff and shift 
    #print('local indx {}, item index {}, weight {}, value {}'.format(indx, sorted_indices[indx], sorted_weight[indx], \
    #                                                                 sorted_value[indx]))
    item_indx = sorted_indices[indx]
    #if indx-1 >0 and indx+1 < len(sorted_weight):
    #    print('local indx {}, weight before {}, weight after {}'.format(indx, sorted_weight[indx-1], sorted_weight[indx+1]))
    if indx-1 >0 and indx+1 < len(sorted_density):
        value_ratio = float(sorted_value[indx-1])/float(sorted_value[indx+1])
        #print('\n\nFloor value ratio ', math.floor(value_ratio*10.))
        if math.floor(value_ratio*10.) > 5:
            indx, item_indx = None, None
            #print('\nReturn Set indx and item indx to None')
        #if indx:
            #print('Finit diff : den ratio ', float(sorted_density[indx+1])/float(sorted_density[indx-1]))
            #print('Finit diff : value ratio ', float(sorted_value[indx-1])/float(sorted_value[indx+1]))
            #print('local indx {}, density before {}, density after {}'.format(indx, sorted_density[indx-1], sorted_density[indx+1])) 
            #print('local indx {}, value before {}, value after {}'.format(indx, sorted_value[indx-1], sorted_value[indx+1])) 
    #print('In finite diff Return, indx {}, item index {}, value {}, weight {}'.format(indx, item_indx, sorted_value[indx],\
    #                                                                                 sorted_weight[indx]))
    return indx, item_indx, max_value_diff, max_value_index, max_density_diff, max_density_index # the 2nd is item_index

def compute_fractions_by_weight(value_densities, capacity, items):
    
    """
        Items with small weights are in the first part of the list items
        Items with weight > capacity are removed
        low_value_density_omit the fraction of low value density remove of the beginging of indices
    """
    print('WEight Heurictis')
    def resort_indices(sorted_weight, sorted_value, sorted_density, sorted_indices, capacity):
        
        #print('In resort indices ', sorted_indices)
        resorted_indices = []
        
        max_density_index, max_value_index = -1, -1
        max_density_diff, max_value_diff = 0, 0
        local_indx, item_indx, max_value_diff, max_value_index, max_density_diff, max_density_index \
        = finite_difference_sorted_weight(sorted_weight, sorted_density, sorted_value, sorted_indices)        
        #print('Local index {}, item_index {}'.format(local_indx, sorted_indices[local_indx]))
        #print('\nMax weight index %d  %d' % (max_value_index, max_density_index))
        #print('\nMean value {}, max diff value {}'.format(statistics.mean(sorted_value), max_value_diff))
        if max_value_diff > statistics.mean(sorted_value) and max_value_index == max_density_index:
            #print('\n\n we sort')
            second_index = None
            #print('Weight at max ', sorted_weight[max_value_index])
            for i in range(max_value_index+2, len(sorted_weight)-1, 1):
                #print('Weight at i ', sorted_weight[i])
                #print('Weight at i-1 ', sorted_weight[i-1])
                if capacity - (sorted_weight[max_value_index]+sorted_weight[i])< 0 and \
                capacity - (sorted_weight[max_value_index]+sorted_weight[i-1])>= 0:
                    second_index = i-1
                    #print('Eq 1', capacity - (sorted_weight[max_value_index]+sorted_weight[i]))
                    #print('Eq 2', capacity - (sorted_weight[max_value_index]+sorted_weight[i-1]))
                    #print('Second index {}, weight {}, sorted_index: {}'.format(second_index, sorted_weight[second_index],\
                    #                                            sorted_indices[second_index]))
            if not second_index: second_index = max_value_index+1
            resorted_indices.extend([sorted_indices[second_index], sorted_indices[max_value_index]])
            
            #print('Sorted indices at i,i+1 {}, resorted_indices {} '.format([sorted_indices[second_index], \
            #                                sorted_indices[max_value_index]],resorted_indices))
            
            resorted_indices = resorted_indices + sorted_indices[:max_value_index-1] + sorted_indices[second_index+1:]
        elif local_indx:
            #print('Local index {}, item_index {}'.format(local_indx, sorted_indices[local_indx]))
            resorted_indices = [sorted_indices[i] for i in range(local_indx,-1,-1)] + \
                    [sorted_indices[i] for i in range(local_indx+1,len(sorted_indices),1)]
            
        else:
            resorted_indices = sorted_indices
            
        return resorted_indices
        

    #print('Mean density: %0.4f, Median: %0.4f' %(statistics.median(value_densities), 
    #                                            statistics.mean(value_densities)))
    
    weights = [item.weight for item in items]
    sorted_indices = sorted(range(len(weights)), key=lambda k: weights[k])
    weight_indices = sorted_indices
    #print('\nWeight indices ', weight_indices)
    sorted_indices = sorted(range(len(value_densities)), 
                            key=lambda k: value_densities[k])
    density_indices = sorted_indices
    #print('Density indices ', density_indices)
    
    #print('Sorted Weight ', [weights[i] for i in weight_indices])
    #print('Sorted Density ', [value_densities[i] for i in weight_indices]) 
    #print('Sorted value ', [items[i].value for i in weight_indices])    
    
    #print('\nWeight indices ', weight_indices)
    sorted_weight = [weights[i] for i in weight_indices]
    sorted_value = [items[i].value for i in weight_indices]
    sorted_density = [value_densities[i] for i in weight_indices] 
    weight_indices = resort_indices(sorted_weight, sorted_value, sorted_density, weight_indices,capacity)
    #print('\nAfter resort Weight indices ', weight_indices)
    
    sorted_items = [items[idx] for idx in weight_indices]
    #print('First weight {} value {}'.format(items[weight_indices[0]].weight, items[weight_indices[0]].value))
    room, estimate = capacity, 0.
    behind_items, before_items = [],[]
    ignore_items, other_items = [],[]
    before_fractions = []

    for idx, item in enumerate(sorted_items):

        if item.weight > capacity:
            ignore_items.append(item)
        elif item.weight <= room: # if item weight less than or equal k, select item
            before_fractions.append(1.)
            before_items.append(item)
            estimate = estimate + float(item.value)
            room = room - before_fractions[-1]*float(item.weight)
        elif room > 0: # if item weight > k, but is not full yet
            if len(sorted_items) - idx > len(sorted_items)/2:
                ignore_items.append(item)
            else:                    
                before_fractions.append(float(room)/float(item.weight))# fraction of item that fits
                before_items.append(item)            
                estimate = estimate + before_fractions[-1]*float(item.value)
                room = room - before_fractions[-1]*float(item.weight)
        else:
            other_items.append(item)

    items = before_items+behind_items+other_items+ignore_items
    fractions = before_fractions+[0]*(len(items)-len(before_fractions))
        

    return items, estimate, fractions



In [387]:
Item = namedtuple("Item", ['index', 'value', 'weight'])

def compute_fractions_by_value(capacity, items, capacity_ratio=0.5):
    
    """
        Items with high value are in the first part of the list items
        Items with weight > capacity are removed
    """
    def pick_from_weight_ratio(capacity, items, sorted_indices):     
                
        sorted_items = [items[idx] for idx in sorted_indices]
        sorted_weight = np.asarray([item.weight for item in sorted_items])
        sorted_density = np.asarray([float(item.value)/float(item.weight) for item in sorted_items])
        sorted_neg_val_by_squared_w = -100.*sorted_density/sorted_weight;
        sorted_value = np.asarray([item.value for item in sorted_items])
        
        weight_ratios= sorted_weight/np.mean(sorted_weight)
        max_mean_density_stats = np.max([statistics.mode(sorted_density), np.mean(sorted_density), np.median(sorted_density)])
        densities_flags = sorted_density >  max_mean_density_stats
        mean_weight_indices = np.argmin(np.abs(weight_ratios-1.))
        print('Mean weight indices ', mean_weight_indices)
        mean_weight_index = None
        if mean_weight_indices:
            if mean_weight_indices.size > 1:
                mean_weight_index = mean_weight_indices[0]
            elif np.isscalar(mean_weight_indices):
                mean_weight_index = mean_weight_indices
            else:
                mean_weight_index = mean_weight_indices[0]
        # move from median up
        resorted_indices = []
        sort_by_neg_val_w2 = []
        if mean_weight_index:
            for i in range(mean_weight_index,-1,-1):
                heapq.heappush(sort_by_neg_val_w2, (sorted_neg_val_by_squared_w[i], sorted_indices[i]))
        if sort_by_neg_val_w2:
            print('\nSorted by Value div by Squared Weights HEAP \n', sort_by_neg_val_w2)
            while sort_by_neg_val_w2:
                idx_element = heapq.heappop(sort_by_neg_val_w2)
                print('nidx_element ', idx_element)
                resorted_indices.append(idx_element[1])
        if mean_weight_index: 
            resorted_indices = resorted_indices+sorted_indices[(mean_weight_index+1):]
        else: 
            resorted_indices = sorted_indices
        print('\n----Resort indices after pop heap---\n ', resorted_indices)
            
        room, estimate = capacity, 0.
        behind_items, before_items = [],[]
        ignore_items, other_items = [],[]
        before_fractions = []

        for idx in resorted_indices:

            if items[idx].weight > capacity:
                ignore_items.append(items[idx])
            elif items[idx].weight <= room: # if item weight less than or equal k, select item
                before_fractions.append(1.)
                before_items.append(items[idx])
                estimate = estimate + float(items[idx].value)
                room = room - before_fractions[-1]*float(items[idx].weight)
            elif room > 0: # if item weight > k, but is not full yet
                if len(items) - idx > len(items)/2:
                    ignore_items.append(items[idx])
                else:                    
                    before_fractions.append(float(room)/float(items[idx].weight))# fraction of item that fits
                    before_items.append(items[idx])            
                    estimate = estimate + before_fractions[-1]*float(items[idx].value)
                    room = room - before_fractions[-1]*float(items[idx].weight)
            else:
                other_items.append(items[idx])

        items = before_items+behind_items+other_items+ignore_items
        fractions = before_fractions+[0]*(len(items)-len(before_fractions))
        
        return estimate, items, fractions
    
    def pick_by_sqr_weights_div_value(capacity, items):
        
        sqr_weights_div_value = [float(item.value)/float(item.weight*item.weight) for item in items]
        sorted_indices = sorted(range(len(sqr_weights_div_value)), key=lambda k: sqr_weights_div_value[k], reverse=True)
        sorted_items = [items[i] for i in sorted_indices]
        
        room, estimate = capacity, 0.
        behind_items, before_items = [],[]
        ignore_items, other_items = [],[]
        before_fractions = []

        for idx, item in enumerate(sorted_items):

            if item.weight > capacity:
                ignore_items.append(item)
            elif item.weight > capacity_ratio*capacity:
                behind_items.append(item)
            elif item.weight <= room: # if item weight less than or equal k, select item
                before_fractions.append(1.)
                before_items.append(item)
                estimate = estimate + float(item.value)
                room = room - before_fractions[-1]*float(item.weight)
            elif room > 0: # if item weight > k, but is not full yet
                if len(sorted_items) - idx > len(sorted_items)/2:
                    ignore_items.append(item)
                else:                    
                    before_fractions.append(float(room)/float(item.weight))# fraction of item that fits
                    before_items.append(item)            
                    estimate = estimate + before_fractions[-1]*float(item.value)
                    room = room - before_fractions[-1]*float(item.weight)
            else:
                other_items.append(item)

        items = before_items+behind_items+other_items+ignore_items
        fractions = before_fractions+[0]*(len(items)-len(before_fractions))
        #print('\nItem size {}, items index {}'.format(len(items), [item.index for item in items]) )
        #print('\n Fractions size {}, fraction {}'.format(len(fractions), fractions))        
        
        return estimate, items, fractions    
    
    def pick_less_opt_following_capacity_ratio(capacity, sorted_items, capacity_ratio):
        
        room, estimate = capacity, 0.
        behind_items, before_items = [],[]
        ignore_items, other_items = [],[]
        before_fractions = []

        for idx, item in enumerate(sorted_items):

            if item.weight > capacity:
                ignore_items.append(item)
            elif abs(capacity_ratio-1.) > 1.e-5 and item.weight > capacity_ratio*capacity:
                behind_items.append(item)
            elif item.weight <= room: # if item weight less than or equal k, select item
                before_fractions.append(1.)
                before_items.append(item)
                estimate = estimate + float(item.value)
                room = room - before_fractions[-1]*float(item.weight)
            elif room > 0: # if item weight > k, but is not full yet
                if len(sorted_items) - idx > len(sorted_items)/2:
                    ignore_items.append(item)
                else:                    
                    before_fractions.append(float(room)/float(item.weight))# fraction of item that fits
                    before_items.append(item)            
                    estimate = estimate + before_fractions[-1]*float(item.value)
                    room = room - before_fractions[-1]*float(item.weight)
            else:
                other_items.append(item)

        items = before_items+behind_items+other_items+ignore_items
        fractions = before_fractions+[0]*(len(items)-len(before_fractions))
        #print('\nItem size {}, items index {}'.format(len(items), [item.index for item in items]) )
        #print('\n Fractions size {}, fraction {}'.format(len(fractions), fractions))        
        
        return estimate, items, fractions
        
    def pick_according_to_density(capacity, sorted_items, sorted_indices):     
                
        sorted_weight = [item.weight for item in sorted_items]
        sorted_density = [float(item.value)/float(item.weight) for item in sorted_items]
        sorted_value = [item.value for item in sorted_items]
        #print('Sorted Weight ', sorted_weight)
        #print('Sorted Density ', sorted_density) 
        #print('Sorted value ', sorted_value)      
        
        room, estimate = capacity, 0.
        behind_items, before_items = [],[]
        ignore_items, other_items = [],[]
        before_fractions = []              
        start_selected_index = None
        max_weigh_in_top_ten, max_density_in_top_ten = False, False
        for k in range(10):
            if sorted_weight[k] == max(sorted_weight): max_weigh_in_top_ten = True
            if sorted_density[k] == max(sorted_density): max_density_in_top_ten = True
                
        if len(sorted_weight) > 10 and max_weigh_in_top_ten and max_density_in_top_ten: # meaning large weight -> large value -> high density

            diff_max_density = [density-max(sorted_density) for density in sorted_density]
            diff_max_weight_ratio = [ (float(weight)-float(max(sorted_weight)))/float(capacity) for weight in sorted_weight]
            non_pos_diff_max_density = [ diff <= 0 for diff in diff_max_density]
            non_pos_diff_max_weight = [diff <= 0 for diff in diff_max_weight_ratio]
            if all(non_pos_diff_max_density) and all(non_pos_diff_max_weight): # meaning density and weight monotonically decreases
            #if any(non_pos_diff_max_density) and any(non_pos_diff_max_weight):
                
                for j, weight_ratio in enumerate(diff_max_weight_ratio):
                    if abs(weight_ratio) > 0.1:
                        start_selected_index = j
                        break   
             
                if start_selected_index and start_selected_index > 0 : 
                    behind_items = sorted_items[:start_selected_index-1]

                for idx in range(start_selected_index, len(sorted_items), 1):    
                    if sorted_items[idx].weight > capacity:
                        ignore_items.append(sorted_items[idx])
                    elif sorted_items[idx].weight <= room: # if item weight less than or equal k, select item
                        before_fractions.append(1.)
                        before_items.append(sorted_items[idx])
                        estimate = estimate + float(sorted_items[idx].value)
                        room = room - before_fractions[-1]*float(sorted_items[idx].weight)
                    elif room > 0: # if item weight > k, but is not full yet
                        if len(sorted_items) - idx > len(sorted_items)/2:
                            ignore_items.append(sorted_items[idx])
                        else:                        
                            before_fractions.append(float(room)/float(sorted_items[idx].weight))# fraction of item that fits
                            before_items.append(sorted_items[idx])            
                            estimate = estimate + before_fractions[-1]*float(sorted_items[idx].value)
                            room = room - before_fractions[-1]*float(sorted_items[idx].weight)
                    else:
                        other_items.append(sorted_items[idx])
                                       
                items = before_items+other_items+ignore_items+behind_items
                fractions = before_fractions+[0]*(len(sorted_items)-len(before_fractions))
                #print('\nItem size {}, items index {}'.format(len(items), [item.index for item in items]) )
                #print('\n Fractions size {}, fraction {}'.format(len(fractions), fractions))        
        
                return estimate, items, fractions
        return None, None, None
                                       
    values = [item.value for item in items]
    sorted_indices = sorted(range(len(values)), key=lambda k: values[k], reverse=True)
    
    
    
    sorted_items = [items[idx] for idx in sorted_indices]
    fractions = None
    estimate = None
    
    estimate, returned_items, fractions = pick_according_to_density(capacity, sorted_items, sorted_indices)
    if estimate and returned_items and fractions:
        #print('\n Value Heuristic Use pick_according_to_density------------------------') 
        items = returned_items
    else:
        #print('\n Value Heuristic Use pick_less_opt_following_capacity_ratio------------------------')  
        #estimate, items, fractions = pick_from_weight_ratio(capacity, items, sorted_indices)
        estimate, items, fractions = pick_less_opt_following_capacity_ratio(capacity, sorted_items, capacity_ratio)
        #estimate, items, fractions =  pick_by_sqr_weights_div_value(capacity, items)
          
  
    return items, estimate, fractions




In [370]:
def solve_bb_lds_dfs(input_data):
    # Modify this code to run your optimization algorithm
    
    # parse the input
    lines = input_data.split('\n')
    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = float(firstLine[1])

    items = []
    value_densities = []
    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, float(parts[0]), float(parts[1])))
        value_densities.append(float(parts[0])/float(parts[1]))
    num_items = len(items)
    #print('Values densities ', value_densities)
    #print('Items ', items)
    #print('Std value densities ', statistics.stdev(value_densities))

    # compute a heuristic: an optimistic estimate of the value: fractions and estimate are of type floating point
    if statistics.stdev(value_densities) < 0.01:
        #print('\nHeuristic by WEight')
        items, estimate, fractions = compute_fractions_by_weight(value_densities, capacity, items)
    else:
        #print('\nHeuristic by Value density')
        items, estimate, fractions = compute_fractions_by_value(capacity, items)
        #items, estimate, fractions = compute_fractions_by_value_capacity_differences(value_densities, capacity, items)
    
    # search the tree for best value
    sys.setrecursionlimit(2000)
    if num_items < 2000: 
        best_node = limited_discrepancy_search(estimate, items, fractions, capacity)
    else: 
        best_node =  depth_first_search(fractions, items, capacity, estimate)
       
    best_value = best_node.solution(num_items)[0]
    best_x_path = best_node.solution(num_items)[1]+[0]*(num_items-len(best_node.solution(num_items)[1]))

    # prepare the solution in the specified output format
    output_data = str(best_value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, best_x_path))
    
    return output_data

In [322]:
def solve_bb_dfs(input_data):
    # Modify this code to run your optimization algorithm
    

    # parse the input
    lines = input_data.split('\n')
    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []
    value_densities = []
    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))
        value_densities.append(float(parts[0])/float(parts[1]))
    num_items = len(items)
    
    items, estimate, fractions = compute_fractions_by_value(capacity, items)
    
    best_node = depth_first_search(fractions, items, capacity, estimate)
    best_value = best_node.solution(num_items)[0]
    best_x_path = best_node.solution(num_items)[1]+[0]*(num_items-len(best_node.solution(num_items)[1]))

    # prepare the solution in the specified output format
    output_data = str(best_value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, best_x_path))
    return output_data

In [ ]:
def solve_greedy_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')
    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])

    items = []

    for i in range(1, item_count+1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i-1, int(parts[0]), int(parts[1])))

    # a trivial greedy algorithm for filling the knapsack
    # it takes items in-order until the knapsack is full
    value = 0
    weight = 0
    taken = [0]*len(items)

    for item in items:
        if weight + item.weight <= capacity:
            taken[item.index] = 1
            value += item.value
            weight += item.weight
    
    # prepare the solution in the specified output format
    output_data = str(value) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))
    return output_data

In [388]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_10000_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
#print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) #1,012,574


 Value Heuristic Use pick_less_opt_following_capacity_ratio------------------------
Adjust cap ratio
BB-LDS:  438250 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [376]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_400_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) 


 Value Heuristic Use pick_according_to_density------------------------
BB-LDS:  3966328 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0

 Value Heuristic Use pick_according_to_density------------------------
BB-DFS :  1063332 0
0 0 0 0 0 0 0 0 0

In [377]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_1000_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) 


 Value Heuristic Use pick_less_opt_following_capacity_ratio------------------------
BB-LDS:  107767 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [378]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_30_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) 

WEight Heurictis
BB-LDS:  99764 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0

 Value Heuristic Use pick_according_to_density------------------------
BB-DFS :  97076 0
0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0
Greedy :  90000 0
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [379]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_50_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) 


 Value Heuristic Use pick_according_to_density------------------------
BB-LDS:  141960 0
0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0

 Value Heuristic Use pick_according_to_density------------------------
BB-DFS :  128071 0
0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Greedy :  140034 0
1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1


In [380]:
file_location = 'C:/D/coursera/discrete_opt/knapsack/data\\ks_200_0'
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print('BB-LDS: ', solve_bb_lds_dfs(input_data))
print('BB-DFS : ', solve_bb_dfs(input_data))
print('Greedy : ', solve_greedy_it(input_data)) 

WEight Heurictis
BB-LDS:  100062 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0

 Value Heuristic Use pick_less_opt_following_capacity_ratio------------------------
BB-DFS :  69144 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
Greedy :  90001 0
1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [58]:
def write_csv_line(filename, *args):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(args)

In [59]:
data_folder = 'C:/D/coursera/discrete_opt/knapsack/data'
data_files = None
for dirpath, dirnames, filenames in os.walk(data_folder):
    data_files = [os.path.join(dirpath, fname) for fname in filenames if fname[:2] == 'ks' and fname[-3:] != 'txt'\
                 and fname[-3:] != 'put']
    
filepaths = list(reversed(data_files))
print('File paths \n', filepaths)

File paths 
 ['C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_2', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_lecture_dp_1', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_82_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_60_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_50_1', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_50_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_500_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_4_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_45_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_40_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_400_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_30_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_300_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_200_1', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_200_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_19_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_106_0', 'C:/D/coursera/discrete_opt/knapsack/data\\ks_100_2', 'C:/D/cour

In [60]:
for fpath in filepaths:
    
    # output file 
    output_filename = os.path.join(data_folder, ''.join([os.path.split(fpath)[-1], '_output']))
    
    with open(fpath, 'r') as input_data_file:
        input_data = input_data_file.read()
    
    print('Input filename: ', os.path.split(fpath)[-1])
    
    # BB-LDS
    start_time = time.time()
    bb_lds_result = solve_bb_lds_dfs(input_data)
    end_time = time.time()
    bb_lds_time = end_time - start_time
    bb_lds_result = bb_lds_result.split('\n')
    print('BB-LDS: {}, Time: {} '.format(bb_lds_result[0], bb_lds_time))
    
    
    #BB-DFS
    start_time = time.time()
    bb_dfs_result = solve_bb_dfs(input_data)
    end_time = time.time()
    bb_dfs_time = end_time - start_time
    bb_dfs_result = bb_dfs_result.split('\n')
    print('BB-DFS: {}, Time: {} '.format(bb_dfs_result[0], bb_dfs_time))
    
    # Greedy
    start_time = time.time()
    greedy_result = solve_bb_dfs(input_data)
    end_time = time.time()
    greedy_time = end_time - start_time
    greedy_result = greedy_result.split('\n')
    print('Greedy: {}, Time: {} '.format(greedy_result[0], greedy_time))
    
    write_csv_line(output_filename, *('methods', 'value' ,'runtime'))
    write_csv_line(output_filename, *('BB-LDS', bb_lds_result[0], bb_lds_time))
    write_csv_line(output_filename, *('BB-DFS', bb_dfs_result[0], bb_dfs_time))
    write_csv_line(output_filename, *('Greedy', greedy_result[0], greedy_time))    
    
    write_csv_line(output_filename, *('BB-LDS\n', bb_lds_result[1]))
    write_csv_line(output_filename, *('BB-DFS\n', bb_dfs_result[1]))
    write_csv_line(output_filename, *('Greedy\n', greedy_result[1]))   
    

Input filename:  ks_lecture_dp_2
BB-LDS: 35 0, Time: 0.0010035037994384766 
BB-DFS: 28 0, Time: 0.0 
Greedy: 28 0, Time: 0.0 
Input filename:  ks_lecture_dp_1
BB-LDS: 11 0, Time: 0.0 
BB-DFS: 11 0, Time: 0.0010058879852294922 
Greedy: 11 0, Time: 0.0 
Input filename:  ks_82_0
BB-LDS: 101447269 0, Time: 0.007018566131591797 
BB-DFS: 94701568 0, Time: 0.0010051727294921875 
Greedy: 94701568 0, Time: 0.0010039806365966797 
Input filename:  ks_60_0
BB-LDS: 90000 0, Time: 0.0030066967010498047 
BB-DFS: 90000 0, Time: 0.0 
Greedy: 90000 0, Time: 0.0 
Input filename:  ks_50_1
BB-LDS: 5300 0, Time: 0.001003265380859375 
BB-DFS: 2752 0, Time: 0.003008127212524414 
Greedy: 2752 0, Time: 0.0020046234130859375 
Input filename:  ks_50_0
BB-LDS: 126289 0, Time: 0.001003265380859375 
BB-DFS: 96601 0, Time: 0.0 
Greedy: 96601 0, Time: 0.0 
Input filename:  ks_500_0
BB-LDS: 54377 0, Time: 0.0010027885437011719 
BB-DFS: 31611 0, Time: 0.0954890251159668 
Greedy: 31611 0, Time: 0.1209874153137207 
Input 

In [ ]:
"""
(K-sorted_weight').*sorted_density'
"""
def compute_fractions_by_value_capacity_differences(value_densities, capacity, items):
    
    densities  = [(capacity-items[i].weight)*value_densities[i] for i in range(len(items))]
    
    sorted_indices = sorted(range(len(densities)), key=lambda k: densities[k], reverse=True)
    items = [items[idx] for idx in sorted_indices]
    
    print('Sorted calculations ', densities)
    print('Sorted Weight ', [items[i].weight for i in sorted_indices])
    print('Sorted Density ', [value_densities[i] for i in sorted_indices]) 
    print('Sorted value ', [items[i].value for i in sorted_indices])      
    
    fractions, room = [0.]*len(items), float(capacity)
    estimate = 0.
    num_omit = 0 # number of items weight > capacity and we remove them
    
    for idx, item in enumerate(items):

        if item.weight > capacity:
            num_omit = num_omit+1
            continue
        elif item.weight <= room: # if item weight less than or equal k, select item
            fractions[idx] = 1.
            estimate = estimate + float(item.value)   
        elif room > 0: # if item weight > k, but is not full yet
            fractions[idx] = float(room)/float(item.weight) # fraction of item that fits
            estimate = estimate + fractions[idx]*float(item.value)

        room = room - fractions[idx]*float(item.weight)

    if num_omit: 
        del items[:num_omit]
        del fractions[:num_omit]

    return items, estimate, fractions

